# テキストマイニングによるTwitter個人アカウントの性格推定
## Character estimation of Twitter personal account by text mining

- 2018-09-17 PyCon JP 2018
- @jumpyoshim ([Twitter](https://twitter.com/jumpyoshim)/[Qiita](https://qiita.com/jumpyoshim)/[GitLab](https://gitlab.com/jumpyoshim)/[はてなブログ](https://jumpyoshim.hatenablog.com/)) / 吉村　潤平
- iRidge, Inc. - Gold Sponsor

<div align="center">
<img src="images/kataomoi_woman.png">
</div>

<div align="center">
<img src="images/pose_atama_kakaeru_woman.png">
</div>

# 「あなたのこともっと知りたい...」
## “ I want to know more about you... ”

<div align="center">
<img src="images/Python_logo_and_wordmark.svg">
</div>

# 目次（INDEX）
- ツイートの収集（Collection of tweets）
- 文書のベクトル化（Vectorization of sentences）
   - 形態素解析（Morphological analysis）
   - 特徴ベクトル（Feature vector）
- 文書分類器(Document classifier)
   - 機械学習（Machine learning）

# ツイートの収集（Collection of tweets）

[python-twitter - PyPI](https://pypi.org/project/python-twitter/)
---
> A Python wrapper around the Twitter API. 

In [1]:
import twitter
from ipython_secrets import get_secret

CONSUMER_KEY = get_secret('CONSUMER_KEY')
CONSUMER_SECRET = get_secret('CONSUMER_SECRET')
ACCESS_TOKEN_KEY = get_secret('ACCESS_TOKEN_KEY')
ACCESS_TOKEN_SECRET = get_secret('ACCESS_TOKEN_SECRET')

api = twitter.Api(
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    access_token_key=ACCESS_TOKEN_KEY,
    access_token_secret=ACCESS_TOKEN_SECRET
)

tweets = api.GetUserTimeline(screen_name='jumpyoshim', count=2)
tweets

[Status(ID=1041532636303249408, ScreenName=jumpyoshim, Created=Mon Sep 17 03:41:45 +0000 2018, Text='RT @CardinalXaro: #pyconjp 株式会社アイリッジのブースでは、簡単なアンケートにお答えいただいた方にZen of Pythonのシールをお渡ししています。 https://t.co/3ODYhlI9hB'),
 Status(ID=1040568453109673984, ScreenName=jumpyoshim, Created=Fri Sep 14 11:50:26 +0000 2018, Text='草\nhttps://t.co/PcNYOgfz8Z')]

# 文書のベクトル化（Vectorization of sentences）

[Bag-of-Words（BoW）](https://tifana.ai/words/natural-language-processing/9302.html)
---
> ベクトル表現の一種。 文章に単語が含まれるかどうかのみを考え、単語の並び方などは考慮しない形式のこと。

## 形態素解析（Morphological analysis）

[形態素解析 - Wikipedia](https://ja.wikipedia.org/wiki/%E5%BD%A2%E6%85%8B%E7%B4%A0%E8%A7%A3%E6%9E%90)
---
> 形態素解析（けいたいそかいせき、Morphological Analysis）とは、文法的な情報の注記の無い自然言語のテキストデータ（文）から、対象言語の文法や、辞書と呼ばれる単語の品詞等の情報にもとづき、形態素（Morpheme, おおまかにいえば、言語で意味を持つ最小単位）の列に分割し、それぞれの形態素の品詞等を判別する作業である

In [2]:
import MeCab

text = 'すもももももももものうち'

tagger = MeCab.Tagger('mecabrc')
print(tagger.parse(text))

すもも	名詞,一般,*,*,*,*,すもも,スモモ,スモモ
も	助詞,係助詞,*,*,*,*,も,モ,モ
もも	名詞,一般,*,*,*,*,もも,モモ,モモ
も	助詞,係助詞,*,*,*,*,も,モ,モ
もも	名詞,一般,*,*,*,*,もも,モモ,モモ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
うち	名詞,非自立,副詞可能,*,*,*,うち,ウチ,ウチ
EOS



[mecab-python3 - PyPI](https://pypi.org/project/mecab-python3/)
---
> This python wrapper for mecab works on both python3.x and python2.x.

In [3]:
import MeCab

tagger = MeCab.Tagger('mecabrc')
data = []
for tweet in tweets:
    node = tagger.parseToNode(tweet.text)
    words = []
    while node:
        meta = node.feature.split(',')
        if meta[0] == '名詞':
            words.append(node.surface.lower())
        node = node.next
    data.append(words)
print(data)

[['rt', '@', 'cardinalxaro', ':', '#', 'pyconjp', '株式会社', 'アイリッジ', 'ブース', '簡単', 'アンケート', 'お答え', '方', 'zen', 'of', 'python', 'シール', '渡し', 'https', '://', 't', '.', 'co', '/', '3', 'odyhli', '9', 'hb'], ['草', 'https', '://', 't', '.', 'co', '/', 'pcnyogfz', '8', 'z']]


## 特徴ベクトル（Feature vector）

[gensim - PyPI](https://pypi.org/project/gensim/)
---
> Gensim is a Python library for topic modelling, document indexing and similarity retrieval with large corpora. Target audience is the natural language processing (NLP) and information retrieval (IR) community.

In [4]:
from gensim import corpora, matutils

dictionary = corpora.Dictionary(data)
data_train = []
for datum in data:
    bow = dictionary.doc2bow(datum)
    dense = list(matutils.corpus2dense([bow], num_terms=len(dictionary)).T[0])
    data_train.append(dense)
print(data_train)

[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0]]


# 文書分類器（Document classifier）

[エゴグラム - Wikipedia](https://ja.wikipedia.org/wiki/%E3%82%A8%E3%82%B4%E3%82%B0%E3%83%A9%E3%83%A0)
---
> エゴグラム (Egogram) とは、エリック・バーン (Eric Berne) の交流分析における自我状態をもとに、弟子であるジョン・M・デュセイ (John M. Dusay) が考案した性格診断法で、**人の心を5つに分類し、その5つの自我状態が放出する心的エネルギーの高さをグラフにしたもののこと** である


- CP(Critical Parent)：支配性
- NP(Nurturing Parent)：寛容性
- A(Adult)：論理性
- FC(Free Child)：奔放性
- AC(Adapted Child)：順応性

Paper test
---
- 5要素のそれぞれに関する質問が10問程度ずつ、全部で50問程度の質問に答えていき、最後にそれを集計し、点数化したものを、CP、NP、A、FC、ACの順に点数をもとに点を打っていき、グラフにする

<div align="center">
<img src="images/egogram.png">
</div>

# [Classification pattern](http://www.egogram-f.jp/seikaku/aisyou/patan.htm)
- ネクラ厭世タイプ（Ｗ型）
- 明朗楽観タイプ（Ｍ型）
- 優柔不断タイプ（Ｎ型）
- ハイパワータイプ（逆Ｎ型）
- 頭でっかちタイプ（逆Ｖ型）
- お手あげタイプ（Ｖ型）
- 典型的ネクラタイプ（Ｕ型）
- ぼんぼんタイプ（逆Ｕ型）
- 頑固オヤジタイプ（左上がり型）
- ガキ丸出しタイプ（右上がり型）
- ハイレベルタイプ（オールＡ型）

- 中庸タイプ（オールＢ型）
- 原始人タイプ（オールＣ型）
- ルーズタイプ（ＣＰ欠乏型）
- クールタイプ（ＮＰ欠乏型）
- 現実無視タイプ（Ａ欠乏型）
- 自閉症タイプ（ＦＣ欠乏型）
- 気ままタイプ（ＡＣ欠乏型）
- 口うるさタイプ（ＣＰ型）
- お人好しタイプ（ＮＰ型）
- コンピュータタイプ（Ａ型）
- 自由奔放タイプ（ＦＣ型）
- 自己卑下タイプ（ＡＣ型）

## 機械学習（Machine learning）

[scikit-learn](https://pypi.org/project/scikit-learn/)
---
> scikit-learn is a Python module for machine learning built on top of SciPy and distributed under the 3-Clause BSD license.

In [6]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()
label_train = ['ネクラ厭世タイプ（Ｗ型）', '明朗楽観タイプ（Ｍ型）']
clf.fit(data_train, label_train)
clf.predict(data_train)

array(['ネクラ厭世タイプ（Ｗ型）', '明朗楽観タイプ（Ｍ型）'], dtype='<U12')

# [@PyConJ](https://twitter.com/PyConJ)の性格を推定してみよう
## Let's estimate personality of @PyConJ

<div align="center">
<img src="images/pyconj.png">
</div>

In [ ]:
from sklearn.externals import joblib

from lib.twitter import get_user_timeline

# ツイートの収集
tweets = get_user_timeline(api, 'PyConJ')

# 形態素解析（名詞の抽出）
node = tagger.parseToNode(tweets)
words = []
while node:
    meta = node.feature.split(',')
    if meta[0] == '名詞':
        words.append(node.surface.lower())
    node = node.next
    
# ベクトル化
dictionary = corpora.Dictionary.load_from_text('Dictionary.txt')
bow = dictionary.doc2bow(words)
dense = list(matutils.corpus2dense([bow], num_terms=len(dictionary)).T[0])

# 機械学習
clf = joblib.load('egogram.pkl')
clf.predict([dense]) 

<div align="center">
<img src="images/seikou_banzai_woman.png">
</div>